<a href="https://colab.research.google.com/github/callmesukhi/BFVM19PROG1/blob/main/assessments/Assignment_week_03_code_review_fenna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 03 Assignment Covid

Welcome to **week three** of this course programming 1. You will learn about combining data with pandas and numpy and you will learn to visualize with bokeh. Concretely, you will preprocess the partly Synthetic Covid data in an appropiate format in order to conduct statistical and visual analysis. Learning outcomes:

- Load a tabular dataset
- Inspect the dataset for quality and metadata information
- Combine data from several tables into one dataframe
- Subselect specific data from dataframes
- Reshape the dataset into a format suitable for visual and statistical analysis
- Perform visual analysis


Your job is to **visualize the lab values taken for COVID-19 patients of survived versus not survived patients**. 

The assignment consists of 6 parts:

- [part 1: load the data](#0)
     - [Exercise 1.1](#ex-11)
- [part 2: data wrangling](#1)
     - [Exercise 2.1](#ex-21)
- [part 3: inspect the data](#2)
     - [Exercise 3.1](#ex-31)
- [part 4: visualize the data](#3)
     - [Exercise 4.1](#ex-41)
- [part 5: plot patient location](#5)
     - [Exercise 6.1](#ex-51)


Part 1 and 4 are mandatory, part 5 is optional (bonus)
To pass the assingnment you need to a score of 60%. 


## About the data

The data is generated by Synthea's COVID-19 module. The data was constructed using three peer-reviewed publications published in the early stages of the global pandemic, when less was known, along with emerging resources, data, publications, and clinical knowledge. The simulation outputs synthetic Electronic Health Records (EHR), including the daily consumption of Personal Protective Equipment (PPE) and other medical devices and supplies. For this assignment the `conditions`, `patients`, `observations`, `careplans` and `encounters` table will be used. The Data is stored in separate tables to avoid redundancy, with as a concequence that tables need to be combined and reorganized in dataframes for analysing purpose.

Source: Walonoski J, Klaus S, Granger E, Hall D, Gregorowicz A, Neyarapally G, Watson A, Eastman J. Synthea™ Novel coronavirus (COVID-19) model and synthetic data set. Intelligence-Based Medicine. 2020 Nov;1:100007. https://doi.org/10.1016/j.ibmed.2020.100007

#### Covid Patients
Patients are considered Covid patients if they are identified with `CODE` `840539006`


#### Survivors
Patients that had covid and where tested negative after isolation have tested code `94531-1`,  SARS-CoV-2 RNA Pnl Resp NAA+probe (covid-sars test) + a value of `Not detected (qualifier value)`. These patients are considered to be survived covid patients. Mind you, a view of these patients died after the isolation treatment, but these are considered not covid related. 

#### Non-Survivors
Patients that did not survived Covid have a `DEATHDATE` which is not null. Mind you that non deaths could be tested postive!

#### Lab values  COVID-19 patients

Patients are monitored for blood and heart conditions once they are admitted in Hospital or under treatment. The lab values of interest are as follow: 

- `48065-7`  Fibrin D-dimer FEU [Mass/volume] in Platelet poor plasma
- `26881-3`   Interleukin 6 [Mass/volume] in Serum or Plasma
- `2276-4` Ferritin [Mass/volume] in Serum or Plasma
- `89579-7` Troponin I.cardiac [Mass/volume] in Serum or Plasma by High sensitivity method
- `731-0` Lymphocytes [#/volume] in Blood by Automated count
- `14804-9` Lactate dehydrogenase [Enzymatic activity/volume] in Serum or Plasma by Lactate to pyruvate reaction


---

<a name='0'></a>
## Part 1: Load the data (20 pt)

Instructions: Load the data of the following files

- conditions.csv
- patients.csv
- observations.csv
- careplans.csv
- encounters.csv

Get yourself familiar with the data. Create some meaningful overviews. Answer the following questions

1. How many patients are there
2. How many covid-patients are there
3. How many patients do have a 'Hospital admission for isolation' encounter
    
<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
    <ul><li>use a unique dataframe for each file, use a meaningful name</li>
    <li>pandas.read_csv() method can be used to read a csv file</li>
    <li>pandas.DataFrame.head() method is often used to inspect the dataframe</li>
    <li>.unique() returns a list of unique values of a column</li>
</ul>
</details>

<a name='ex-11'></a>
### 1.1 Code your solution

In [ ]:
#YOUR solution

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import datetime

In [4]:
conditions = pd.read_csv("/content/conditions.csv") 
patients = pd.read_csv("/content/patients.csv") 
observations = pd.read_csv("/content/observations.csv") 
care_plans = pd.read_csv("/content/careplans.csv") 
encounters = pd.read_csv("/content/encounters.csv") 

In [5]:
print(f'There are {len(patients.Id.unique())} patients in total')

There are 12352 patients in total


In [6]:
covid_conditions = conditions[conditions.CODE == 840539006]
print(f'There are {len(covid_conditions)} covid patients')

There are 8820 covid patients


In [7]:
# How many patients do have a 'Hospital admission for isolation' encounter
admitted = encounters[encounters.CODE == 1505002].PATIENT.unique()
print(f'There are {len(admitted)} admitted patients')

There are 1867 admitted patients


In [8]:
survived = patients.DEATHDATE.notna().sum()
print(f'{survived} patients survived')



2352 patients survived


### 1.2 Test your solution
The following function needs to be called. You can use this as a test. There are however more meaningful overviews 
you can create. 

In [ ]:
def part1(num_pat, num_cov, num_admitted, num_died):
    print(f'There are {num_pat} patients in total')
    print(f'There are {num_cov} covid patients')
    print(f'There are {num_admitted} admitted patients')
    print(f'{num_died} patients died')


### Expected outcome

There are 12352 patients in total
There are 8820 covid patients
There are 1867 admitted patients
2352 patients died

---

<a name='1'></a>
## Part 2: Data Wrangling: set up the dataframe (30 pt)

In this part we are going to combine data to create a dataframe with values of interest for the lab values analysis. 

We would like a dataframe containing the following information per record (only Covid patients!!!)

- `PATIENT` - the ID of the covid patient
- `days` - the number of days the patient is under observation
- `CODE-Y` - the code of the observation  
- `VALUE` - the lab value of the observation

where only the following observation codes needs to be selected:

- `48065-7`  Fibrin D-dimer FEU [Mass/volume] in Platelet poor plasma
- `26881-3`   Interleukin 6 [Mass/volume] in Serum or Plasma
- `2276-4` Ferritin [Mass/volume] in Serum or Plasma
- `89579-7` Troponin I.cardiac [Mass/volume] in Serum or Plasma by High sensitivity method
- `731-0` Lymphocytes [#/volume] in Blood by Automated count
- `14804-9` Lactate dehydrogenase [Enzymatic activity/volume] in Serum or Plasma by Lactate to pyruvate reaction

The days information is not primarely available and needs to be calculated by substracting observation DATE - START. 

An example of such a dataframe is given below:

                                      days   CODE-Y  VALUE    UNITS
PATIENT                                                            
00079a57-24a8-430f-b4f8-a1cf34f90060   0.0  14804-9  241.8      U/L
00079a57-24a8-430f-b4f8-a1cf34f90060   0.0   2276-4  489.5     ug/L
00079a57-24a8-430f-b4f8-a1cf34f90060   0.0  48065-7    0.4    ug/mL
00079a57-24a8-430f-b4f8-a1cf34f90060   0.0    731-0    1.1  10*3/uL
00079a57-24a8-430f-b4f8-a1cf34f90060   0.0  89579-7    3.4    pg/mL
00079a57-24a8-430f-b4f8-a1cf34f90060   1.0    731-0    1.1  10*3/uL
00079a57-24a8-430f-b4f8-a1cf34f90060   2.0  14804-9  233.7      U/L
00079a57-24a8-430f-b4f8-a1cf34f90060   2.0   2276-4  497.3     ug/L
00079a57-24a8-430f-b4f8-a1cf34f90060   2.0  48065-7    0.3    ug/mL
00079a57-24a8-430f-b4f8-a1cf34f90060   2.0    731-0    1.0  10*3/uL

In [ ]:
#Possible approach:

#Select all the patients with covid from the conditions table
#Combine conditions table (only covid patients) with the patient table into a covid_patient table
#select the only the relevant lab observations from the observations table into a lab_obs table
#merge the covid_patient table with the lab_obs table into a covid_patients_obs table
#clean the covid_patients_obs table (rename columns, select only relevant columns, sort, typecast, add days column)

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
    <ul><li>you can use pandas.DataFrame.merge() to merge dataframes</li>
    <li>df = df[(df.CODE == condition1 | df.CODE == condition1 )] selects rows with CODE of 2 conditional values</li>
    <li>df.DATE - df.START return days if DATE and START are datetime format</li>
    <li>pd.to_datetime() can be used to typecast to datetime</li>
</ul>
</details>

<a name='ex-21'></a>
### 2.1 Code your solution

In [ ]:
#solution

---

In [12]:
#First let select all the patients with covid. 
covid_conditions = conditions[conditions.CODE == 840539006]
covid_conditions.head(2)

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
4,2020-03-01,2020-03-30,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,681c380b-3c84-4c55-80a6-db3d9ea12fee,840539006,COVID-19
11,2020-03-13,2020-04-14,067318a4-db8f-447f-8b6e-f2f61e9baaa5,1ea74a77-3ad3-4948-a9cc-3084462035d6,840539006,COVID-19


In [13]:
#Then combine this with the patients records to get information 
covid_patients = covid_conditions.merge(patients, how='left', left_on='PATIENT', right_on='Id')
covid_patients.head(2)

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,2020-03-01,2020-03-30,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,681c380b-3c84-4c55-80a6-db3d9ea12fee,840539006,COVID-19,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,2017-08-24,NaN,999-68-6630,NaN,NaN,NaN,Jacinto644,Kris249,NaN,NaN,NaN,white,nonhispanic,M,Beverly Massachusetts US,888 Hickle Ferry Suite 38,Springfield,Massachusetts,Hampden County,1106.0,42.151961,-72.598959,8446.49,1499.08
1,2020-03-13,2020-04-14,067318a4-db8f-447f-8b6e-f2f61e9baaa5,1ea74a77-3ad3-4948-a9cc-3084462035d6,840539006,COVID-19,067318a4-db8f-447f-8b6e-f2f61e9baaa5,2016-08-01,NaN,999-15-5895,NaN,NaN,NaN,Alva958,Krajcik437,NaN,NaN,NaN,white,nonhispanic,F,Boston Massachusetts US,1048 Skiles Trailer,Walpole,Massachusetts,Norfolk County,2081.0,42.177370,-71.281353,89893.40,1845.72


In [14]:
#select the relevant lab observations
lab_obs = observations[(observations.CODE == '48065-7') | (observations.CODE == '26881-3') | 
                          (observations.CODE == '2276-4') | (observations.CODE == '89579-7') |
                          (observations.CODE == '2532-0') | (observations.CODE == '731-0') |
                          (observations.CODE == '14804-9')
                      ]
lab_obs.head(2)

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,VALUE,UNITS,TYPE
273,2020-02-19,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,e97e8d37-7497-4c13-98fd-a4a45655c0bb,731-0,Lymphocytes [#/volume] in Blood by Automated c...,1.1,10*3/uL,numeric
292,2020-02-19,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,e97e8d37-7497-4c13-98fd-a4a45655c0bb,48065-7,Fibrin D-dimer FEU [Mass/volume] in Platelet p...,0.4,ug/mL,numeric


In [15]:
#clean the lab_obs
lab_obs = lab_obs.drop('ENCOUNTER', axis=1)
lab_obs = lab_obs.drop('TYPE', axis = 1)
d = {'CODE':'CODE-Y'}
lab_obs = lab_obs.rename(columns = d, inplace = False)
lab_obs.head(2)

,DATE,PATIENT,CODE-Y,DESCRIPTION,VALUE,UNITS
273,2020-02-19,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,731-0,Lymphocytes [#/volume] in Blood by Automated c...,1.1,10*3/uL
292,2020-02-19,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,48065-7,Fibrin D-dimer FEU [Mass/volume] in Platelet p...,0.4,ug/mL


In [16]:
#merge lab observations with covid_patients
covid_patients_obs = covid_patients.merge(lab_obs, on='PATIENT')
covid_patients_obs.head(2)

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION_x,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,DATE,CODE-Y,DESCRIPTION_y,VALUE,UNITS
0,2020-02-19,2020-02-28,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,e3143bce-4a59-40aa-a198-7a9e54077fd8,840539006,COVID-19,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,2019-06-12,NaN,999-81-4349,NaN,NaN,NaN,Karyn217,Mueller846,NaN,NaN,NaN,white,nonhispanic,F,New Bedford Massachusetts US,570 Abshire Forge Suite 32,Colrain,Massachusetts,Franklin County,NaN,42.74022,-72.722648,22940.0,893.28,2020-02-19,731-0,Lymphocytes [#/volume] in Blood by Automated c...,1.1,10*3/uL
1,2020-02-19,2020-02-28,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,e3143bce-4a59-40aa-a198-7a9e54077fd8,840539006,COVID-19,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,2019-06-12,NaN,999-81-4349,NaN,NaN,NaN,Karyn217,Mueller846,NaN,NaN,NaN,white,nonhispanic,F,New Bedford Massachusetts US,570 Abshire Forge Suite 32,Colrain,Massachusetts,Franklin County,NaN,42.74022,-72.722648,22940.0,893.28,2020-02-19,48065-7,Fibrin D-dimer FEU [Mass/volume] in Platelet p...,0.4,ug/mL


In [17]:
#derive days
covid_patients_obs['START'] = pd.to_datetime(covid_patients_obs.START)
covid_patients_obs['DATE'] = pd.to_datetime(covid_patients_obs.DATE)
covid_patients_obs['lab_days'] = covid_patients_obs.DATE - covid_patients_obs.START
covid_patients_obs['days'] = covid_patients_obs.lab_days / np.timedelta64(1, 'D')
#make value numeric
covid_patients_obs['VALUE'] = pd.to_numeric(covid_patients_obs['VALUE'], errors='coerce')
covid_patients_obs.head(2)

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION_x,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,DATE,CODE-Y,DESCRIPTION_y,VALUE,UNITS,lab_days,days
0,2020-02-19,2020-02-28,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,e3143bce-4a59-40aa-a198-7a9e54077fd8,840539006,COVID-19,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,2019-06-12,NaN,999-81-4349,NaN,NaN,NaN,Karyn217,Mueller846,NaN,NaN,NaN,white,nonhispanic,F,New Bedford Massachusetts US,570 Abshire Forge Suite 32,Colrain,Massachusetts,Franklin County,NaN,42.74022,-72.722648,22940.0,893.28,2020-02-19,731-0,Lymphocytes [#/volume] in Blood by Automated c...,1.1,10*3/uL,0 days,0.0
1,2020-02-19,2020-02-28,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,e3143bce-4a59-40aa-a198-7a9e54077fd8,840539006,COVID-19,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,2019-06-12,NaN,999-81-4349,NaN,NaN,NaN,Karyn217,Mueller846,NaN,NaN,NaN,white,nonhispanic,F,New Bedford Massachusetts US,570 Abshire Forge Suite 32,Colrain,Massachusetts,Franklin County,NaN,42.74022,-72.722648,22940.0,893.28,2020-02-19,48065-7,Fibrin D-dimer FEU [Mass/volume] in Platelet p...,0.4,ug/mL,0 days,0.0


In [18]:
columns_to_keep = ['PATIENT', 'CODE-Y', 'DESCRIPTION_y', 'VALUE', 'UNITS', 'days','DEATHDATE', 'STOP']
covid_patients_obs = covid_patients_obs[columns_to_keep]

In [19]:
covid_patients_obs.head(2)

,PATIENT,CODE-Y,DESCRIPTION_y,VALUE,UNITS,days,DEATHDATE,STOP
0,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,731-0,Lymphocytes [#/volume] in Blood by Automated c...,1.1,10*3/uL,0.0,NaN,2020-02-28
1,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,48065-7,Fibrin D-dimer FEU [Mass/volume] in Platelet p...,0.4,ug/mL,0.0,NaN,2020-02-28


In [20]:
covid_patients_obs = covid_patients_obs.sort_values(['PATIENT', 'days', 'CODE-Y'])
covid_patients_obs = covid_patients_obs[['PATIENT', 'days', 'CODE-Y', 'VALUE', 'UNITS']]
covid_patients_obs.head(3)

,PATIENT,days,CODE-Y,VALUE,UNITS
48611,00079a57-24a8-430f-b4f8-a1cf34f90060,0.0,14804-9,241.8,U/L
48609,00079a57-24a8-430f-b4f8-a1cf34f90060,0.0,2276-4,489.5,ug/L
48608,00079a57-24a8-430f-b4f8-a1cf34f90060,0.0,48065-7,0.4,ug/mL


<a name='2'></a>
## Part 3: Data Wrangling, split into survived and not survived (10 pt)

Now we have the required data we would like to split the data into survived and not survived. First we fetch all the ids of the survived and deceased patients. We can use these ids to select the records of the survived patients and the patients that did not survived

Your job is to split the data into survived and not survived records. There are multiple ways to do this. One way is the  `.isin()` method

In [21]:
#the following code is given, RUN THIS CELL
#get survived and deceased ids
completed_isolation_patients = care_plans[(care_plans.CODE == 736376001) & (care_plans.STOP.notna()) \
                                          & (care_plans.REASONCODE == 840539006)].PATIENT
negative_covid_patient_ids = observations[(observations.CODE == '94531-1') \
                                          & (observations.VALUE == 'Not detected (qualifier value)')].PATIENT.unique()
survivor_ids = np.union1d(completed_isolation_patients, negative_covid_patient_ids)
deceased_ids = patients[patients.DEATHDATE.notna()].Id

<a name='ex-31'></a>
### 3.1 Code your solution

In [ ]:
#YOUR CODE HERE

### 3.2 Test your solution

In [22]:
covid_patients_obs_survived = covid_patients_obs[covid_patients_obs.PATIENT.isin(survivor_ids)]
covid_patients_obs_deceased = covid_patients_obs[covid_patients_obs.PATIENT.isin(deceased_ids)]

In [24]:
def test3(survived, died):
    print(f'patients records survived: {survived}, patients records deceased {died}')
#call the test3

#### Expected outcome

patients records survived: 57303, patients records deceased 16793

---

In [25]:
test3(len(covid_patients_obs_survived), len(covid_patients_obs_deceased))

patients records survived: 57303, patients records deceased 16793


<a name='3'></a>
## Part 4: Plot the data (20 pt)

Create plots with the lab data, for each code one plot. Separate the survivors and the deceased by color. An example of such a plot is given below. You can create 6 plots in one grid (for each code one plot) or use the widgets to select a code. Plot on the x-axis the days, on the y-axis the VALUE. Use proper labels, titles and legends.


<a name='ex-41'></a>
### 4.1 Code your solution

In [ ]:
#possible solution

In [26]:
code_to_display = {'48065-7': 'D-dimer', 
                   '2276-4': 'Serum Ferritin',
                    '89579-7': 'High Sensitivity Cardiac Troponin I',
                    '26881-3': 'Interleukin 6 in Serum or Plasma', 
                    '731-0': 'Lymphocytes',
                    '14804-9': 'Lactate dehydrogenase'}

In [27]:
print(code_to_display.keys())

dict_keys(['48065-7', '2276-4', '89579-7', '26881-3', '731-0', '14804-9'])


In [28]:
from bokeh.plotting import figure, show
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_notebook
output_notebook()

In [29]:
def make_data(df, code):
    data = df[df['CODE-Y']==code]
    return data

def make_datapoints(p, data, color, legend_label):
    p.circle(data.days,data.VALUE, color=color, legend_label = legend_label, size = 8, alpha = 0.5) 
    p.legend.location = "top_left"
    return p

def make_plot(title, y):
    p = figure(title=title, background_fill_color="#fafafa")
    p.xaxis.axis_label = 'days'
    p.yaxis.axis_label = y
    p.grid.grid_line_color="white"
    return p

def make_panel(code):
    data_s = make_data(covid_patients_obs_survived, code)
    data_d = make_data(covid_patients_obs_deceased, code)
    pt = make_plot(code_to_display[code], 'pg/mL')
    pt = make_datapoints(pt, data_s, 'green', 'survived')
    pt = make_datapoints(pt, data_d, 'red', 'deceased')
    
    ##### HERE IS THE PANEL PART #######

    panel = Panel(child=pt, title=code_to_display[code][0:11])
    
    ####################################
    return panel

panels = [make_panel(i) for i in code_to_display.keys()]
tabs = Tabs(tabs=panels)
show(tabs)


<a name='4'></a>
## Part 5: Plot the location of the patients (10 pt)

This is a bonus part. Can you plot the patients location on a map? See also 
https://docs.bokeh.org/en/latest/docs/user_guide/geo.html
